In [1]:
import folium
import os
import csv
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import shapefile
import pickle
import random

In [27]:
# get speed an crash data
with open('Dane_speed_dict.pk', 'rb')as f:
    speed_dict = pickle.load(f)
    
with open('Dane_distance_dict.pk', 'rb')as f:
    distance_dict = pickle.load(f)
    
# load functional class file
fnct_cls_path = os.path.join("..","..","transportation_data","dane-county","dane_wislr_rdwy_link_fnct_cls.csv")
with open(fnct_cls_path,'r') as f:
    fnct_cls = list(csv.reader(f))
fnct_header = fnct_cls[0]
fnct_cls = fnct_cls[1:]

fnct_cls_list = []
for r in fnct_cls:
    fnct_cls_list.append(int(r[fnct_header.index('RDWY_LINK_ID')]))

In [37]:
# read ref position
ref_path = os.path.join("..","..","transportation_data","dane-county","Dane_County_Shape_file","Dane_Ref_Coord.shp")
ref = shapefile.Reader(ref_path)

ref_dict = {}
for r in ref.shapeRecords():
    ref_position = np.array([r.shape.points[:][0][1],r.shape.points[:][0][0]])
    ref_dict[r.record[1]] = ref_position

In [38]:
fnct_cls_list = list(set(fnct_cls_list))

road_to_ref = {}
for r in fnct_cls:
    road_to_ref[int(r[fnct_header.index('RDWY_LINK_ID')])] = \
                        np.array([int(r[fnct_header.index('REF_SITE_FROM_ID')]),int(r[fnct_header.index('REF_SITE_TO_ID')])]) 

In [57]:
# AADT information
AADT = {'Rural Local': 200,
        'Rural Major Collector': 1600,
        'Rural Principal Arterial': 5400,
        'Rural Minor Arterial' : 2400,
        'Rural Minor Collector' : 700,
        'Urban Collector' : 2300,
        'Urban Local' : 400,
        'Urban Minor Arterial' : 5300,
        'Urban Other Connecting Link Principal Arterial':11500
       }

In [10]:
# read crash information
crash_path = os.path.join("..","..","transportation_data","dane-county","dane_crash_link_postspd_2017_2020.csv")
with open(crash_path,'r') as f:
    crash_file = list(csv.reader(f))
    
crash_header = crash_file[0]
crash_file = crash_file[1:]

In [26]:
# initialize crash count
crash_dict = {}
for key in speed_dict:
    crash_dict[key] = 0
   
for c in crash_file:
    if int(c[crash_header.index('WISLR_LINKID')]) in crash_dict:
        crash_dict[int(c[crash_header.index('WISLR_LINKID')])] += 1
        count += 1

33716

In [34]:
# AADT_DICT

AADT_dict = {}
for key in speed_dict:
    AADT_dict[key] = None
    
has_AADT = []
not_has_AADT = []
for fc in fnct_cls:
    if int(fc[fnct_header.index('RDWY_LINK_ID')]) in crash_dict and fc[fnct_header.index('FNCT_CLS_CTGY_TYDC')] != '':
        AADT_dict[int(fc[fnct_header.index('RDWY_LINK_ID')])] = fc[fnct_header.index('FNCT_CLS_CTGY_TYDC')]
        has_AADT.append(int(fc[fnct_header.index('RDWY_LINK_ID')]))

not_has_AADT = set(speed_dict.keys()) - set(has_AADT)

In [47]:
# get road coord
def calculate_middle(ref1,ref2):
    return (ref_dict[ref1] + ref_dict[ref2])/2

def calculate_distance(road1,road2):
    return sum(abs(road_coord[road1] - road_coord[road2]))

road_coord = {}
# 1. get the coordinate of all the roads
for key in speed_dict:
    startAndEnd = road_to_ref[key]
    if startAndEnd[0] in ref_dict and startAndEnd[1] in ref_dict:
        road_coord[key] = calculate_middle(startAndEnd[0],startAndEnd[1])
        
has_AADT = list(set([x for x in has_AADT if x in road_coord]))
not_has_AADT = [x for x in not_has_AADT if x in road_coord]

In [51]:
idx = 0 
for road1 in not_has_AADT:
    if idx % 100 == 0:
        print(idx)
    road_id_temp = None
    dist = None
    for road2 in has_AADT:
        if road_id_temp == None and dist == None:
            road_id_temp = road2
            dist = calculate_distance(road1,road2)
        else:
            if dist < calculate_distance(road1,road2):
                road_id_temp = road2
                dist = calculate_distance(road1,road2)
                
    AADT_dict[road1] = AADT_dict[road_id_temp]
    idx += 1 

0
100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300
2400
2500
2600
2700
2800
2900
3000
3100
3200
3300
3400
3500


In [54]:
for i in AADT_dict:
    if AADT_dict[i] == None:
        print(i)

In [73]:
risk_rate = {}

for key in speed_dict:
    risk_rate[key] = 100000000*crash_dict[key]/(365*4*AADT[AADT_dict[key]]*distance_dict[key])

In [74]:
# deal with no crash data
AADT_min = {}
for key in AADT:
    AADT_min[key] = None
    
for key in risk_rate:
    if risk_rate[key] > 0:
        AADT_temp = AADT_dict[key]
        if AADT_min[AADT_temp] == None:
            AADT_min[AADT_temp] = risk_rate[key]
        elif risk_rate[key] < AADT_min[AADT_temp]:
            AADT_min[AADT_temp] = risk_rate[key]
            

In [75]:
for key in risk_rate:
    if risk_rate[key] <= 0:
        AADT_temp = AADT_dict[key]
        risk_rate[key] = AADT_min[AADT_temp]

In [71]:
adjusted_risk = {}

for key in risk_rate:
    adjusted_risk[key] = risk_rate[key] * distance_dict[key]

In [72]:
adjusted_risk

{4849664: 36130.13698630137,
 4849665: 135616.43835616438,
 4849666: 63356.16438356164,
 4849668: 54280.821917808214,
 4849669: 63356.16438356164,
 4849670: 27054.794520547945,
 4849671: 45205.479452054795,
 4849672: 36130.13698630137,
 4849673: 45205.479452054795,
 4849674: 36130.13698630137,
 4849675: 27054.794520547945,
 4849676: 72260.27397260274,
 4849678: 25163.787969029185,
 4849679: 72260.27397260274,
 4849686: 17301.96545562835,
 4849687: 45205.479452054795,
 4849688: 45205.479452054795,
 4849689: 54280.821917808214,
 4849690: 45205.479452054795,
 4849691: 63356.16438356164,
 4849692: 36130.13698630137,
 4849693: 72260.27397260274,
 4849694: 45205.479452054795,
 4849695: 18150.68493150685,
 4849696: 72260.27397260274,
 4849697: 72260.27397260274,
 4849698: 18150.68493150685,
 4849699: 27054.794520547945,
 4849700: 27054.794520547945,
 4849701: 3776.0571768910067,
 4849703: 171746.57534246575,
 4849704: 171746.57534246575,
 4849706: 226027.39726027395,
 4849707: 81335.616438356